In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Desription,Rain (inches),Snow (inches)
0,0,Majene,ID,-3.54,118.97,78.19,86,100,10.51,overcast clouds,0.00,0
1,1,Albany,US,42.60,-73.97,84.00,76,75,7.78,broken clouds,0.00,0
2,2,Port Alfred,ZA,-33.59,26.89,57.96,64,94,28.32,light rain,0.36,0
3,3,Flic En Flac,MU,-20.27,57.36,72.00,68,75,12.75,shower rain,0.00,0
4,4,Batsfjord,NO,70.63,29.72,53.60,66,75,10.29,broken clouds,0.00,0


In [3]:
city_data_df.drop(columns=['Unnamed: 0'])
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Desription,Rain (inches),Snow (inches)
0,0,Majene,ID,-3.54,118.97,78.19,86,100,10.51,overcast clouds,0.00,0
1,1,Albany,US,42.60,-73.97,84.00,76,75,7.78,broken clouds,0.00,0
2,2,Port Alfred,ZA,-33.59,26.89,57.96,64,94,28.32,light rain,0.36,0
3,3,Flic En Flac,MU,-20.27,57.36,72.00,68,75,12.75,shower rain,0.00,0
4,4,Batsfjord,NO,70.63,29.72,53.60,66,75,10.29,broken clouds,0.00,0


In [4]:
# Configure gmaps to use API key.
gmaps.configure(api_key=g_key)

In [5]:
city_data_df.dtypes

Unnamed: 0              int64
City                   object
Country                object
Lat                   float64
Lng                   float64
Max Temp              float64
Humidity                int64
Cloudiness              int64
Wind Speed            float64
Current Desription     object
Rain (inches)         float64
Snow (inches)           int64
dtype: object

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Raining?
rain_q = (input("Do you want it to be raining? (yes/no) "))
# Snowing?
snow_q = (input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 89
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [7]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain (inches)"] == 0 if rain_q == "no" else city_data_df["Rain (inches)"] > 0) &
                                       (city_data_df["Snow (inches)"] == 0 if snow_q == "no" else city_data_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Desription,Rain (inches),Snow (inches)
0,0,Majene,ID,-3.54,118.97,78.19,86,100,10.51,overcast clouds,0.0,0
1,1,Albany,US,42.60,-73.97,84.00,76,75,7.78,broken clouds,0.0,0
7,7,Sapouy,BF,11.55,-1.77,84.40,64,100,7.02,overcast clouds,0.0,0
13,13,Itarema,BR,-2.92,-39.92,85.48,59,46,13.87,scattered clouds,0.0,0
14,14,Ambulu,ID,-8.35,113.61,76.01,88,100,5.44,overcast clouds,0.0,0
19,19,Quang Ngai,VN,15.12,108.80,79.29,88,100,4.61,overcast clouds,0.0,0
29,29,Uruzgan,AF,32.83,66.00,79.45,17,0,7.29,clear sky,0.0,0
30,30,Ponta Do Sol,PT,32.67,-17.10,78.80,57,20,20.80,few clouds,0.0,0
37,37,Tongren,CN,27.72,109.19,75.99,81,65,2.64,broken clouds,0.0,0
41,41,Los Llanos De Aridane,ES,28.66,-17.92,77.00,69,20,16.11,few clouds,0.0,0


In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, description and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Desription", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Desription,Lat,Lng,Hotel Name
0,Majene,ID,78.19,overcast clouds,-3.54,118.97,
1,Albany,US,84.00,broken clouds,42.60,-73.97,
7,Sapouy,BF,84.40,overcast clouds,11.55,-1.77,
13,Itarema,BR,85.48,scattered clouds,-2.92,-39.92,
14,Ambulu,ID,76.01,overcast clouds,-8.35,113.61,
19,Quang Ngai,VN,79.29,overcast clouds,15.12,108.80,
29,Uruzgan,AF,79.45,clear sky,32.83,66.00,
30,Ponta Do Sol,PT,78.80,few clouds,32.67,-17.10,
37,Tongren,CN,75.99,broken clouds,27.72,109.19,
41,Los Llanos De Aridane,ES,77.00,few clouds,28.66,-17.92,


In [9]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Desription,Lat,Lng,Hotel Name
0,Majene,ID,78.19,overcast clouds,-3.54,118.97,Kost Putri RABBANI
1,Albany,US,84.00,broken clouds,42.60,-73.97,
7,Sapouy,BF,84.40,overcast clouds,11.55,-1.77,Auberge St Louis
13,Itarema,BR,85.48,scattered clouds,-2.92,-39.92,Windvillage
14,Ambulu,ID,76.01,overcast clouds,-8.35,113.61,Hotel Ambulu Jember
19,Quang Ngai,VN,79.29,overcast clouds,15.12,108.80,Central Hotel
29,Uruzgan,AF,79.45,clear sky,32.83,66.00,
30,Ponta Do Sol,PT,78.80,few clouds,32.67,-17.10,Hotel do Campo
37,Tongren,CN,75.99,broken clouds,27.72,109.19,Fanjing Mountain International Conference Center
41,Los Llanos De Aridane,ES,77.00,few clouds,28.66,-17.92,Valle Aridane


In [12]:
# Get the maximum temperature.
max_temp = hotel_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Desription} and {Max Temp}°F</dd>
</dl>
"""

In [14]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# Add a marker layer for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="")

TypeError: 'module' object is not callable